In [1]:
import scipy as sp
import numpy as np; #importing packages we will use
import matplotlib.pyplot as plt;
import pandas as pd;
pd.options.mode.chained_assignment = None  # default='warn'
import multiprocessing as mp
from multiprocessing.pool import ThreadPool as Pool
from scipy.io import loadmat
import os 
import scipy.linalg as la
import time
from scipy.optimize import minimize
import pickle
import time
import concurrent.futures
import cProfile
import random as rd
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor, as_completed
from joblib import Parallel,delayed
import dask.array as da
from dask.distributed import Client
import h5py
import pstats
import copy
import statsmodels.api as sm
from linearmodels.iv import IV2SLS


###importing data dictionaries I will be using 
os.chdir('G:\\Greg\\full_model')
with open('main_dictionary_2019.pkl', 'rb') as fp:
    main_dictionary = pickle.load(fp)
with open('useful_variables.pkl', 'rb') as fp:
    usable_county_years = pickle.load(fp)

### Visualization of the matrix I will be using to interact demographics with individuals

$$
\begin{pmatrix}
t_2 & t_3 & nh\_black & hispanic 
\end{pmatrix}
\begin{pmatrix}
\theta_{2,int} & \theta_{2,p} & \theta_{2,cs} & \theta_{2,d} & \theta_{2,v} & \theta_{2,h}\\
\theta_{3,int} & \theta_{3,p} & \theta_{3,cs} & \theta_{3,d} & \theta_{3,v} & \theta_{3,h}\\
\theta_{b,int} & \theta_{b,p} & \theta_{b,cs} & \theta_{b,d} & \theta_{b,v} & \theta_{b,h}\\
\theta_{hi,int} & \theta_{hi,p} & \theta_{hi,cs} & \theta_{hi,d} & \theta_{hi,v} & \theta_{hi,h}\\
\end{pmatrix}
\begin{pmatrix}
1 \\ p \\ cs \\ d \\ v \\h
\end{pmatrix}
$$

* Plan/individual interactions needs to be a 4x6 matrix


In [4]:
###using initial guess for parameters based on a one-county model I made 

with open('initial_estimates_county_2019.pkl', 'rb') as fp:
    old_guess = pickle.load(fp)

### Functions to calculate Model Predicted Market Shares and Plan choice probabilities for every individual

In [8]:
#takes in individual-level data, plan-level data, an additional guess for the mean uitility values, and the parameter values
# calculates the model predicted market share of each plan in each county
def mkt_share_testing_b(individual_data_1, individual_data_2, plan_data, lin_util_val, guess_util):
    
    ##preparing parameter guesses
    guess_util_1 = guess_util[0:24].reshape(4,-1) #first 24 parameters into a 4x6 matrix
    guess_util_2 = guess_util[24:]
    
    #gets the plan_attributes for each plan in the county (and including an intercept)
    ones_array = np.ones((plan_data.shape[0],1))
    ptemp = np.hstack((ones_array, plan_data))
    
    #takes calculates nonlinear utility for each individual for all combinations of plans in each county
    nonlin_guess1 = individual_data_1@(guess_util_1)@(ptemp.T)
    nonlin_guess2a = individual_data_2@(guess_util_2.reshape(-1,1))
    nonlin_guess2 = np.broadcast_to(nonlin_guess2a, (nonlin_guess2a.shape[0], plan_data.shape[0]))
    nonlin_guess = nonlin_guess1 + nonlin_guess2
    
   #Creating a linear utility matrix for all individuals, for each plan
    lin_util_vec = lin_util_val.reshape((-1,1)).T
    lin_vec_full = np.tile(lin_util_vec, (nonlin_guess.shape[0],1)) #broadcasts lin_util_vec to all rows of individuals
    
    #adding linear and nonlinear utility for each plan
    full_log_util = nonlin_guess + lin_vec_full 
    
    #calculating individual choice probability of each plan
    temp1 = np.zeros((len(full_log_util[:,0]),1))
    full_log_util2 = np.hstack((temp1, full_log_util))
    lsumexp = sp.special.logsumexp(full_log_util2, axis=1, keepdims=True)
    lsumexp2 = np.tile(lsumexp, (1,len(full_log_util2[1])))
    predicted_choice_probability = np.exp(full_log_util2 - lsumexp2)[:,1:] #getting the pcp of all non-outside_option plans
    
    #calculating model predicted share of all MA plans 
    model_predicted_share = (1/len(predicted_choice_probability))*predicted_choice_probability.sum(axis=0)

    return model_predicted_share

In [10]:
#takes in individual-level data, plan-level data, an additional guess for the mean uitility values, and the parameter values
# calculates the model predicted probability each individual chooses each plan in a county

def pcp_matrix_county_b(temp_ind_1, temp_ind_2, ptemp1, lin_util_val, guess_util):
    
    ##preparing parameter guesses
    guess_util_1 = guess_util[0:24].reshape(4,-1)
    guess_util_2 = guess_util[24:]
    
    plan_vars = ['eff_price_adjusted', 'costsharing_pmpm_adjusted','dental_coverage_indicator', 
                 'eyewear_coverage_indicator', 'hearing_aides_coverage_indicator']

    #gets the plan_attributes for each plan in the county (and including an intercept)
    ones_array = np.ones((ptemp1.shape[0],1))
    ptemp = np.hstack((ones_array, ptemp1))
    
   #takes calculates nonlinear utility for each individual for all combinations of plans in each county
    nonlin_guess1 = temp_ind_1@(guess_util_1)@(ptemp.T)
    nonlin_guess2a = temp_ind_2@(guess_util_2.reshape(-1,1))
    nonlin_guess2 = np.broadcast_to(nonlin_guess2a, (nonlin_guess2a.shape[0], ptemp1.shape[0]))
    nonlin_guess = nonlin_guess1 + nonlin_guess2
    
   #Creating a linear utility matrix for all individuals, for each plan
    lin_util_vec = lin_util_val.reshape((-1,1)).T
    lin_vec_full = np.kron(np.ones((len(nonlin_guess),1)),lin_util_vec) #broadcasts lin_util_vec to all rows of individuals
    
    #adding linear and nonlinear utility for each plan
    full_log_util = nonlin_guess + lin_vec_full 
    
    #calculating choice probabilities using logsumexp trick
    temp1 = np.zeros((len(full_log_util[:,0]),1))
    full_log_util2 = np.hstack((temp1, full_log_util))
    lsumexp = sp.special.logsumexp(full_log_util2, axis=1)
    lsumexp2 = np.kron(lsumexp.reshape(-1,1),np.ones((1, full_log_util2.shape[1])))
    predicted_choice_probability_full = np.exp(full_log_util2 - lsumexp2)

    return predicted_choice_probability_full

## Contraction Mapping

In [12]:
def contraction_mapping_b(data_dictionary, usable_county_years, guess_util, tol):
    
    data_dictionary2 = copy.deepcopy(data_dictionary)
    
    # solves contraction mapping county by county
    for county in usable_county_years:
        ##get individual data and plan_data to use
        plan_vars = ['eff_price_adjusted', 'costsharing_pmpm_adjusted','dental_coverage_indicator', 
                     'eyewear_coverage_indicator', 'hearing_aides_coverage_indicator']
        
        #gets the individual data
        temp_ind_1 = data_dictionary2[county]['individual_data'][['income_tercile_2.0', 'income_tercile_3.0',
                                                                 'nh_black', 'hispanic']].values
        temp_ind_2 = data_dictionary2[county]['individual_data'][['employer_plan','h_age','h_age_sq','healthcode_1',
                                                                 'healthcode_2', 'healthcode_3', 'healthcode_5']].values
        #gets the plan_attributes for each plan in the county
        ptemp1 = data_dictionary2[county]['plan_data'][plan_vars].values
        
        #gets the initial linear utility values from berry1994 and calculates the predicted county shares
        test1 = data_dictionary2[county]['plan_data']['lin_util_init'].values
        guess_mkt_share = mkt_share_testing_b(temp_ind_1, temp_ind_2, ptemp1, test1, guess_util)


        #gets true market share values to use in contraction mapping
        s = data_dictionary2[county]['plan_data']['plan_market_share'].values
        #temp_data['model_share'] = guess_mkt_share

        #solves for mean utility values (in test2) that match model predicted shares to observed market shares in actual data
        #using contraction mapping
        eps = 1000
        while eps > tol:
            bob = mkt_share_testing_b(temp_ind_1, temp_ind_2, ptemp1,test1, guess_util)

            if np.isnan(bob).any():
                return "Bad Parameter Guess"

            test2 = test1 + np.log(s/bob)

            eps = np.max(np.abs(test2-test1))
            test1 = test2.copy()

        data_dictionary2[county]['plan_data']['mean_util_guess'] = test2
        
    #returns updated dictionary with the new average utilities
    return data_dictionary2

## Likelihood function

In [14]:
def likelihood_function_b(dict,usable_county_years,guess_util):
    
    dict1 = dict.copy()
    
    #calculating sample size
    N=0
    likelihood_vec = np.array([])
    
    
    plan_vars = ['eff_price_adjusted', 'costsharing_pmpm_adjusted','dental_coverage_indicator', 'eyewear_coverage_indicator', 
                 'hearing_aides_coverage_indicator']
    
    #sums the weighted log-likelihood county by county
    for county in usable_county_years:
        
        ##gets individual data for use in the likelihood function
        weights = dict1[county]['individual_data']['weight_scaled'].values
        guess_lin_util = dict1[county]['plan_data']['mean_util_guess'].values
        
        temp_ind_1 = dict1[county]['individual_data'][['income_tercile_2.0', 'income_tercile_3.0',
                                                                 'nh_black', 'hispanic']].values
        temp_ind_2 = dict1[county]['individual_data'][['employer_plan','h_age','h_age_sq','healthcode_1',
                                                                 'healthcode_2', 'healthcode_3', 'healthcode_5']].values


        #gets the plan_attributes for each plan in the county
        ptemp_1 = dict1[county]['plan_data'][plan_vars].values
        
        pcp_full = pcp_matrix_county_b(temp_ind_1, temp_ind_2, ptemp_1, guess_lin_util, guess_util)

        m,n = pcp_full.shape
            
        #getting matrix of all plans offered in each county
        plans_matrix  = dict1[county]['plan_data']['merge_variable'].values
        OO = np.array(['OO'])
        plans_matrixOO = np.concatenate((OO,plans_matrix))

        individual_plans = dict1[county]['individual_data']['merge_variable'].values

        shares_vector = np.zeros((m))
        
        #getting model-predicted choice probability for plan individual actually chooses
        for i in range(m):
            index = np.where(individual_plans[i] == plans_matrixOO)[0][0]
            shares_vector[i] = pcp_full[i,index]
        
        #taking logs, weighting observatiosn
        ln_shares = np.log(shares_vector)
        shares_weights = np.multiply(ln_shares, weights)
        sum_ln = np.sum(shares_weights)
        
        #adding to likelihood vector of all counties
        likelihood_vec = np.concatenate([likelihood_vec, np.array([sum_ln])])
        N = N + m
    
    #summing the log likelihood over all counties (and scaling by 1/N)
    likelihood_sum = (1/N)*np.sum(likelihood_vec)
    
    return likelihood_sum

In [204]:
#calculating log-likelihood
start_time = time.perf_counter()

likelihood_info_2= likelihood_function_b(main_dictionary, usable_county_years, old_guess_x)

end_time = time.perf_counter()
total = end_time-start_time
print(f"elapsed time: {total} seconds")

elapsed time: 4.397685099975206 seconds


### Objective Function

In [15]:
def objective_function(theta1_guess, dict, usable_county_years, tol):
    
    dict2 = contraction_mapping_b(dict, usable_county_years, theta1_guess, tol)
    
    if dict2 == "Bad Parameter Guess":
        return 1e30
    
    log_likelihood = likelihood_function_b(dict2, usable_county_years, theta1_guess)
    log_likelihood_new = -log_likelihood
    
    return log_likelihood_new

In [206]:
start_time = time.perf_counter()

objective_function_value = objective_function(old_guess_x, main_dictionary, usable_county_years, 1e-10)

end_time = time.perf_counter()
total = end_time-start_time
print(f"elapsed time: {total} seconds")

elapsed time: 19.488908600003924 seconds


### Maximize Objective function 

In [214]:
start_time = time.time()

##maximizing likelihood function using BFGS method, an initial guess from a toned-down version of the model, and a 
##contraction mapping tolerance of 1e-10
initial_estimates = minimize(objective_function, old_guess_x, args = (main_dictionary, usable_county_years, 1e-10),
                            method = "BFGS", options={'gtol': 1e-4})

end_time = time.time()

##saving estimates to use later 
with open('initial_estimates_full_2019.pkl', 'wb') as f:
    pickle.dump(initial_estimates, f)
    print('estimates successfully saved to file')
    
elapsed_time = (end_time - start_time)/3600

print(f"Elapsed time: {elapsed_time} hours")

estimates successfully saved to file
Elapsed time: 16.93829968591531 hours


In [6]:
###loading the initial parameter estimates

with open('initial_estimates_full_2019.pkl', 'rb') as fp:
    initial_estimates = pickle.load(fp)

In [7]:
###dividing the estimates into the parameters that are demographic/plan interactions and those that are just demographic
# variables

interaction_parameters = (initial_estimates.x[0:24])
interaction_parameters_reshaped = interaction_parameters.reshape(4,-1)
non_interaction_parameters = initial_estimates.x[24:]

### Getting the mean utility guess

In [11]:
###reloading the main dictionary

with open('main_dictionary_2019.pkl', 'rb') as fp:
    main_dictionary_a = pickle.load(fp)

In [22]:
###updating the dataset with the mean-utility values that correspond to MLE Estimates

new_data = contraction_mapping_b(main_dictionary_a, usable_county_years, initial_estimates.x, 1e-10)

In [23]:
##recreating data frame of plan level data to export to stata for other uses


new_data_frame = pd.DataFrame()

for county in usable_county_years:
    temp_df = new_data[county]['plan_data']
    new_data_frame =pd.concat([new_data_frame, temp_df])
    
new_data_frame.to_stata('output_plan_data_2019.dta')

C:\Users\GES58\AppData\Local\Temp\ipykernel_4504\1026565887.py:7: InvalidColumnName: 
Not all pandas column names were valid Stata variable names.
The following replacements have been made:

    eff_premium_instrument_border_adjusted   ->   eff_premium_instrument_border_ad
    premium_instrument_border_adjusted   ->   premium_instrument_border_adjust

If this is not what you expect, please make sure you have Stata-compliant
column names in your DataFrame (strings only, max 32 characters, only
alphanumerics and underscores, no Stata reserved words)

  new_data_frame.to_stata('output_plan_data_2019.dta')


In [121]:
##recreating data frame of individual level data to export to stata for other uses

individual_data_frame = pd.DataFrame()

for county in usable_county_years:
    temp_df_ind = new_data[county]['individual_data']
    individual_data_frame = pd.concat([individual_data_frame, temp_df_ind])

individual_data_frame.to_stata('individual_data_output_2019.dta')

C:\Users\GES58\AppData\Local\Temp\ipykernel_4504\3213672992.py:7: InvalidColumnName: 
Not all pandas column names were valid Stata variable names.
The following replacements have been made:

    income_tercile_1.0   ->   income_tercile_1_0
    income_tercile_2.0   ->   income_tercile_2_0
    income_tercile_3.0   ->   income_tercile_3_0

If this is not what you expect, please make sure you have Stata-compliant
column names in your DataFrame (strings only, max 32 characters, only
alphanumerics and underscores, no Stata reserved words)

  individual_data_frame.to_stata('individual_data_output_2019.dta')


In [25]:
sample_size = individual_data_frame.shape[0]